In [ ]:
# Importing the requried libraries
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [26]:
file_url= "E:\cons\pony\data\market\MarketSales.xlsx"
retail_data = pd.read_excel(file_url) # This may take a couple minutes
retail_data.head()

,ID,ITEMCODE,ITEMNAME,FICHENO,DATE_,AMOUNT,PRICE,LINENETTOTAL,LINENET,BRANCHNR,...,CLIENTCODE,CLIENTNAME,BRANDCODE,BRAND,CATEGORY_NAME1,CATEGORY_NAME2,CATEGORY_NAME3,STARTDATE,ENDDATE,GENDER
0,1.0,8.0,TOZ SEKER,15560,2017-01-02,2.000,2.65,5.30,4.90,14.0,...,6476.0,Hanım CANBULAT,NaN,NaN,İÇECEK,ÇAY KAHVE,SEKER TATLANDIRICI,2017-01-03 09:25:03,2017-01-03 09:25:43,K
1,2.0,20868.0,KIRMIZI MERCIMEK,15560,2017-01-02,1.006,2.80,2.82,2.79,14.0,...,6476.0,Hanım CANBULAT,167,BAKLİYAT,GIDA,BAKLİYAT,AÇIK BAKLİYAT,2017-01-03 09:25:03,2017-01-03 09:25:43,K
2,3.0,8583.0,"TEST MATIK 1,5 KG NORMAL",15560,2017-01-02,1.000,4.95,4.95,4.19,14.0,...,6476.0,Hanım CANBULAT,229,TEST,DETERJAN TEMİZLİK,ÇAMAŞIR YIKAMA,TOZ DETERJAN,2017-01-03 09:25:03,2017-01-03 09:25:43,K
3,4.0,1454.0,BIZIM MAKARNA BONCUK,15560,2017-01-02,1.000,1.10,1.10,1.02,14.0,...,6476.0,Hanım CANBULAT,146,ÜLKER,GIDA,MAKARNA,MAKARNA,2017-01-03 09:25:03,2017-01-03 09:25:43,K
4,5.0,13519.0,FILIZ MAKARNA KISA KESME 500 GR,15560,2017-01-02,1.000,1.10,1.10,1.02,14.0,...,6476.0,Hanım CANBULAT,52,FİLİZ,GIDA,MAKARNA,MAKARNA,2017-01-03 09:25:03,2017-01-03 09:25:43,K


In [61]:
retail_data.info()
cleaned_retail = retail_data.loc[pd.isnull(retail_data.CLIENTCODE) == False]
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611108 entries, 0 to 611107
Data columns (total 26 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   ID              611107 non-null  float64       
 1   ITEMCODE        603941 non-null  float64       
 2   ITEMNAME        603941 non-null  object        
 3   FICHENO         611108 non-null  int64         
 4   DATE_           611107 non-null  datetime64[ns]
 5   AMOUNT          611107 non-null  float64       
 6   PRICE           611107 non-null  float64       
 7   LINENETTOTAL    611107 non-null  float64       
 8   LINENET         611107 non-null  float64       
 9   BRANCHNR        611107 non-null  float64       
 10  BRANCH          611107 non-null  object        
 11  SALESMAN        611108 non-null  object        
 12  CITY            611107 non-null  object        
 13  REGION          611107 non-null  object        
 14  LATITUDE        611107 non-null  flo

In [62]:
item_lookup = cleaned_retail[['ITEMCODE', 'ITEMNAME']].drop_duplicates() # Only get unique item/description pairs
#item_lookup['ITEMCODE'] = item_lookup.ITEMCODE.astype(str) # Encode as strings for future lookup ease
item_lookup.head()

,ITEMCODE,ITEMNAME
0,8.0,TOZ SEKER
1,20868.0,KIRMIZI MERCIMEK
2,8583.0,"TEST MATIK 1,5 KG NORMAL"
3,1454.0,BIZIM MAKARNA BONCUK
4,13519.0,FILIZ MAKARNA KISA KESME 500 GR


In [63]:
item_lookup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9334 entries, 0 to 610758
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ITEMCODE  9333 non-null   float64
 1   ITEMNAME  9333 non-null   object 
dtypes: float64(1), object(1)
memory usage: 218.8+ KB


In [64]:
count = 0
nonint = 0
for val in cleaned_retail['CLIENTCODE']:
    if val is None:
        print("NULL")
    if isinstance(val, int):
        count = count + 1
    else:
        nonint = nonint + 1
print(count)
print(nonint)

0
592743


In [65]:
cleaned_retail['CLIENTCODE'] = cleaned_retail.CLIENTCODE.astype(int) # Convert to int for customer ID
cleaned_retail = cleaned_retail[['ITEMCODE', 'AMOUNT', 'CLIENTCODE']] # Get rid of unnecessary info
cleaned_retail.head()

<ipython-input-65-7145163005d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_retail['CLIENTCODE'] = cleaned_retail.CLIENTCODE.astype(int) # Convert to int for customer ID


,ITEMCODE,AMOUNT,CLIENTCODE
0,8.0,2.000,6476
1,20868.0,1.006,6476
2,8583.0,1.000,6476
3,1454.0,1.000,6476
4,13519.0,1.000,6476


In [66]:
grouped_cleaned = cleaned_retail.groupby(['CLIENTCODE', 'ITEMCODE']).sum().reset_index() # Group together

grouped_cleaned.AMOUNT.loc[grouped_cleaned.AMOUNT == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased # This would have come as zero as the returns are recorded as negatives

grouped_purchased = grouped_cleaned.query('AMOUNT > 0') # Only get customers where purchase totals were positive

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [67]:
# Final dataset
grouped_purchased.head()

,CLIENTCODE,ITEMCODE,AMOUNT
0,1,1615.0,1.000
1,1,2733.0,1.000
2,1,2776.0,1.000
3,1,3780.0,1.000
4,1,3888.0,1.086


In [ ]:
grouped_purchased.to_excel('E:\cons\pony\data\market\MarketSales_grouped.xlsx')

In [69]:
customers = list(np.sort(grouped_purchased.CLIENTCODE.unique())) # Get our unique customers

products = list(grouped_purchased.ITEMCODE.unique()) # Get our unique products that were purchased

quantity = list(grouped_purchased.AMOUNT) # All of our purchases

In [70]:
# Get the associated row indices
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=customers, ordered=False)
rows = grouped_purchased.CLIENTCODE.astype('category', cat_type).cat.codes 

# Get the associated column indices
cat_type_prod = CategoricalDtype(categories=products, ordered=False)
cols = grouped_purchased.ITEMCODE.astype('category', cat_type_prod).cat.codes 

purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [71]:
purchases_sparse

<49263x9333 sparse matrix of type '<class 'numpy.float64'>'
	with 525818 stored elements in Compressed Sparse Row format>

In [72]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.8856349491755

In [73]:
import random

def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    
    random.seed(0) # Set the random seed to zero for reproducibility
    
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered

In [74]:
# 20% of the data has been masked for this exercise
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [75]:
import implicit
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

In [76]:
from sklearn import metrics

def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [77]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [78]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.745, 0.899)

In [79]:
# example
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [86]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    
    return item_lookup.loc[item_lookup.ITEMCODE.isin(prod_codes)]

In [81]:
customers_arr[:5]

array([1, 2, 3, 4, 5], dtype=int64)

In [82]:
from sklearn.preprocessing import MinMaxScaler

In [89]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    #print(item_list)
    #print(item_lookup)
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        #print(code)
        #print(item_lookup.ITEMNAME.loc[item_lookup.ITEMCODE == code])
        rec_list.append([code, item_lookup.ITEMNAME.loc[item_lookup.ITEMCODE == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'ITEMCODE': codes, 'ITEMNAME': descriptions}) # Create a dataframe 
    return final_frame[['ITEMCODE', 'ITEMNAME']] # Switch order of columns around

In [84]:
rec_items(782554, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

[1615. 2733. 2776. ... 8860. 8928. 8988.]
        ITEMCODE                         ITEMNAME
0            8.0                        TOZ SEKER
1        20868.0                 KIRMIZI MERCIMEK
2         8583.0         TEST MATIK 1,5 KG NORMAL
3         1454.0             BIZIM MAKARNA BONCUK
4        13519.0  FILIZ MAKARNA KISA KESME 500 GR
...          ...                              ...
609473   10538.0              A. MAX ANTEP FISTIK
609591    8921.0          PEYMAN KURU ÜZÜM 200 GR
609618   11602.0   DURU DUS JELI 250ML SHINE İNCİ
609994    7771.0      PANTENE Y 400 ML.2. %50 IND
610758   18415.0        YAREN KIOS KATLI TABAKLIK

[9334 rows x 2 columns]
2776.0
1049    KENT OLIPS STIC BAL&LIMON 28GR
Name: ITEMNAME, dtype: object
7168.0
3449    NIVEA LIP DUDAK KORUYUCU MED PROTECTION
Name: ITEMNAME, dtype: object
4828.0
9048    SIGNAL D.F. PERFORMANS
Name: ITEMNAME, dtype: object
1659.0
87042    PASTAVILLA PESTO FESLEGENLI FISTIKLI MAK.SOSU3...
Name: ITEMNAME, dtype: object
15332.0


,ITEMCODE,ITEMNAME
0,2776.0,KENT OLIPS STIC BAL&LIMON 28GR
1,7168.0,NIVEA LIP DUDAK KORUYUCU MED PROTECTION
2,4828.0,SIGNAL D.F. PERFORMANS
3,1659.0,PASTAVILLA PESTO FESLEGENLI FISTIKLI MAK.SOSU3...
4,15332.0,HUNCA BITTIM SABUNU 100 ML
5,247.0,ULKER SAKLIKOY YULAFLI 133GR
6,22047.0,P.BAHCE 52291 ORGANIK BOYALI BARDAK 6'LI
7,5576.0,WELLA KOLESTON KOPUK BOYA 7/0 KUMRAL
8,3706.0,DALIN BEBEK SAMPUANI 1000ML.
9,9825.0,CANBEBE IKIZ 3-6KG KUCUK BOY 66YENİ


In [87]:
# Let's try another example
get_items_purchased(782762, product_train, customers_arr, products_arr, item_lookup)

,ITEMCODE,ITEMNAME
1049,2776.0,KENT OLIPS STIC BAL&LIMON 28GR
2078,3345.0,F SAFF ALUMINYUM FOLYO 15 METRE
4289,16101.0,FIDANLAR KARABIBER 100 GR
5214,2733.0,ANANAS TANE
19291,529.0,CAYKUR DIDI MINI SOGUK CAY 250 ML CILEK
228089,15411.0,MEHTAP TEFLON 3'LU TAVA 2012
252681,13364.0,FINISH TABLET HEPSI 1 ARADA 40 LI


In [90]:
rec_items(782762, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,ITEMCODE,ITEMNAME
0,23338.0,WEE AKIDE DAMAKLI SILIKON EMZIK NO:3
1,20742.0,SOKE TAM BUGDAY UNU 1KG
2,14169.0,OMO MATIK 5KG ACTIVE
3,23489.0,KOSKA SUPER IKILI TAHIN PEKMEZ KARISIMI 200 GR
4,9375.0,IPANA 3 BOYUTLU BEYAZLIK LUXE 75 ML
5,12997.0,BAGDAT KORI 40GR EKO
6,3402.0,PINAR UCGEN PEYNIR SADE 8LI
7,20739.0,SOKE OZEL KEPEK UN 1KG
8,9701.0,PANTENE 400GRKOP DOK SAC SAMP 2012
9,249.0,ULKER PETIBOR PASTA BISK.800 GR


In [92]:
# Let's try another example
get_items_purchased(783199, product_train, customers_arr, products_arr, item_lookup)


,ITEMCODE,ITEMNAME
1723,7815.0,NUTELLA FINDIK EZMESI 180 GR
2218,7577.0,TORKU SADE TAHIN HELVA 500 GR
2544,8572.0,PAPIA CEP MENDIL 3 AL 2 ODE
5214,2733.0,ANANAS TANE
15800,4966.0,SARELLE KAKAOLU FINDIK EZMESI 50 GR
33498,19187.0,DOGADAN YESIL CAY CHAL 20'LI
73025,7639.0,SHE ROLL-ON IS COOL 40ML
81740,15962.0,AKDEM KARADUT PEKMEZI 820 GR
107251,19717.0,TURSIL MATIK 5KG BODRUM BEGONVİLİ
146572,22257.0,PENGUEN FASULYE 820GR TNK


In [93]:
rec_items(783199, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,ITEMCODE,ITEMNAME
0,1099.0,ORKID ULTRA PLATINUM COMFORT 18AD NORMAL
1,14793.0,GLADE NATURE'S FLORAL PERF. 400 ML 2.SI %50
2,6700.0,"LOREAL SUBLIME KOPUK 8,30 ALTIN GUNES SARISI"
3,6392.0,BIZDEN SARI MERCIMEK 1 KG
4,19767.0,GENERAL RENKLI 15W MINI TOP
5,7168.0,NIVEA LIP DUDAK KORUYUCU MED PROTECTION
6,21401.0,COLGATE D.M. GELISMIS BEYAZLIK 125ML+25ML HED.
7,19247.0,DOGADAN EARL GREY 100 LU DEMLIK POSET CAY
8,2776.0,KENT OLIPS STIC BAL&LIMON 28GR
9,2153.0,DR.OETKER HAVUCLU KEK KARISIMI 535 GR
